In [10]:
# import libraries
import numpy as np
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import uuid
import pickle
import ase.io
from ase.optimize import BFGS
import os
from glob import glob
from glob import iglob
from pymatgen.io.vasp.outputs import Outcar
from os import path
import warnings
warnings.filterwarnings("ignore")

### This notebooks is used to update the dataset by adding the energy, forces and scf convergence information into the database. 

In [30]:
## load the mappings stored from previous experiments
with open('dft_settings_study_mappings.pkl', 'rb') as read_file:
    mappings = pickle.load(read_file)
df  = pd.DataFrame(mappings).transpose()
# make a dataframe from the mappings and query for the system of choice
df.rename(columns={'class': 'cat_class'}, inplace=True)
df

,bulk_id,ads_id,bulk_mpid,bulk_symbols,ads_symbols,miller_index,shift,top,cat_class,anomaly,...,xc,oc20_sid,final_energy,unconstrained_atoms_forces,max_nelm,nelm,last_scf_cycle_dE,frame_num,algo,cat_class
5c5e5485d9834e70acae9a70a72d37f1,5986,6,mp-27482,P4S14Ag8,*CH,"[1, 2, 2]",0.266,False,2,0,...,PBE,random2531071,-739.906053,"[[-0.278737, -0.902639, -0.301206], [-0.169526...",60,44,-0.000067,initial,NaN,NaN
0b67c6760b054370a58c02561ea43b3e,5986,6,mp-27482,P4S14Ag8,*CH,"[1, 2, 2]",0.266,False,2,0,...,PBE,random2531071,-740.114336,"[[-0.30608, -0.844158, -0.280684], [-0.166057,...",60,44,-0.000076,initial,NaN,NaN
e328c120fd424020af35db82010c663a,5986,6,mp-27482,P4S14Ag8,*CH,"[1, 2, 2]",0.266,False,2,0,...,PBE,random2531071,-740.285592,"[[-0.321304, -0.825194, -0.275521], [-0.164556...",60,44,-0.000046,initial,NaN,NaN
73f81835306f45dcae5ce3503a201321,5986,6,mp-27482,P4S14Ag8,*CH,"[1, 2, 2]",0.266,False,2,0,...,PBE,random2531071,-740.355678,"[[-0.329061, -0.822248, -0.288398], [-0.167081...",60,44,-0.000062,initial,NaN,NaN
09694fd2ce1946bcab5ffa55659f7f52,5986,6,mp-27482,P4S14Ag8,*CH,"[1, 2, 2]",0.266,False,2,0,...,PBE,random2531071,-740.29936,"[[-0.320361, -0.819488, -0.269843], [-0.162119...",60,43,-0.000097,initial,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39d1f2b35bf942f0bba9f1e584ef506f,640,10,mp-539,S6Ga4,None,"[1, 2, 2]",0.09,False,NaN,1,...,PBE,random1615253,-367.749082,"[[-0.099433, 0.108575, 0.080273], [0.038161, 0...",60,37,-0.000031,initial,NaN,2
2665d7fa3ddc43be8ad59df19f02f989,4294,0,mp-559694,GaMo4S8,None,"[1, 0, 0]",0.242,True,NaN,3,...,PBE,random2183382,-953.010316,"[[0.098244, -0.023472, 0.173263], [-0.315339, ...",60,50,-0.000068,initial,NaN,2
f7de873a97844a3b8c863bce4cfc4729,8430,8,mp-1228454,Al2ZnSe4,None,"[1, 0, 0]",0.437,True,NaN,3,...,PBE,random1638351,-260.850824,"[[-0.000503, 0.191117, 0.029012], [-0.000331, ...",60,26,-0.000092,initial,NaN,2
cab58cc664614c60a5b2681131ae74fd,1474,7,mp-850036,Mn2S2,None,"[0, 0, 1]",0.312,False,NaN,1,...,PBE,random2500067,-439.582344,"[[0.000571, -0.004183, 1.768796], [-0.001679, ...",60,46,0.000038,initial,NaN,2


In [32]:
# pick the unfinished calculations only by querying if they don't have final energy stored
unfinished_calc_df = df.query("final_energy.isnull()==True")

In [26]:
# update the energy, forces and max nelm of each calc
rootdir_glob = '/home/jovyan/shared-scratch/kabdelma/oc20_data_quality_project/oc20_analysis/kpts_error_cancel_study/'
final_energies = {}
unconstrained_atoms_forces = {}
max_nelm = {}
for uuid in tqdm(unfinished_calc_df.index.values):
    try:
        outcar_dir = rootdir_glob + uuid + "/OUTCAR"
        complete_calc = False
        with open(outcar_dir) as outcar_file:
            for line in outcar_file:
                if "Total CPU time used" in line:
                    complete_calc = True
        # check if calculation is complete
        if complete_calc:     
            # extract final energy from the outcar file
            outcar = Outcar(outcar_dir)
            final_energies[uuid] = outcar.final_fr_energy
            # extract forces on the unconstrained atoms of the slab and the adsorbate
            atoms_obj = ase.io.read(outcar_dir)
            forces = atoms_obj._calc.get_forces()
            unconstrained_atoms_forces[uuid] = forces[~atoms_obj._constraints[0].index, :]
            # extract the maximum NELM parameter from the OUTCAR file
            with open(outcar_dir) as outcar:
                nelm_line = [line for line in outcar if line.rfind("NELM")> -1][0]
                max_nelm[uuid] = int(nelm_line.split()[2].replace(';', ''))
        else:
            final_energies[uuid] = None
            unconstrained_atoms_forces[uuid]  = None
            max_nelm[uuid]  = None
            
    except FileNotFoundError as exception:
        # if OUTCAR doesn't exist, put None in place of Energy,Forces, max NELM
        final_energies[uuid] = None
        unconstrained_atoms_forces[uuid]  = None
        max_nelm[uuid]  = None

100%|██████████| 16/16 [00:01<00:00, 15.53it/s]


In [27]:
# update the nelm and the dE of the last scf cycle
nelm = {}
last_scf_cycle_dE = {}
for uuid in tqdm(unfinished_calc_df.index.values):
    try:
        # get the nelm and the final change in energy of the last scf cycle
        with open(rootdir_glob + uuid + '/' + 'vasp.out') as vaspout:
            complete_calc = False
            DAVS = []
            ## checks if the calculation is complete by checking for '1 F' string 
            ## which only apprears at the end of the file
            for line in vaspout:
                if '1 F' in line :
                    complete_calc = True
                if 'DAV' in line:
                    DAVS.append(line) 
            if complete_calc:
                for idx, line in enumerate(DAVS):
                    line = line.split(' ')
                    line = [string for string in line if (string != '')]
                    DAVS[idx] = line
                # delta E is the column with index 3 in vasp.out file
                last_scf_cycle_dE[uuid] = float(DAVS[-1][3])
                nelm[uuid] = len(DAVS)
            # place None if the calculation is incompete yet. 
            else: 
                nelm[uuid] = None
                last_scf_cycle_dE[uuid] = None
                
    # if vasp.out doesn't exist that means that the calculation has not started yet
    except FileNotFoundError as exception:
        nelm[uuid] = None
        last_scf_cycle_dE[uuid] = None

100%|██████████| 16/16 [00:00<00:00, 155.00it/s]


In [28]:
# check if the length if all the vectors being added are equal
assert(len(final_energies)==len(unconstrained_atoms_forces))&(len(final_energies)==len(max_nelm))
assert(len(final_energies)==len(nelm))&(len(final_energies)==len(last_scf_cycle_dE))
# make a copy of the mappings and make the updates on it
updated_mappings = mappings.copy()
for key in final_energies.keys():
    updated_mappings[key]["final_energy"] = final_energies[key]
    updated_mappings[key]["unconstrained_atoms_forces"] = unconstrained_atoms_forces[key]
    updated_mappings[key]["max_nelm"] = max_nelm[key]
    updated_mappings[key]["nelm"] = nelm[key]
    updated_mappings[key]["last_scf_cycle_dE"] = last_scf_cycle_dE[key]

In [29]:
## dump the updated mappings
with open('dft_settings_study_mappings.pkl', 'wb') as fout:
    pickle.dump(updated_mappings, fout)